In [ ]:
import re
import os
import csv
import json
import requests
from urllib import parse
from bs4 import BeautifulSoup
from locals.output_data import exportData
from locals.date_scrape import getDate, lastStamp
from locals.automate import By, Keys, EC, Action, wait_for, genBrowser, driverWait

In [ ]:
# declaring the session

session = requests.Session()

# declaring the header

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

In [ ]:
# get_job function

def get_job(pager, header):
    url = "https://jobs.issworld.com/tile-search-results/?q=&sortColumn=referencedate&sortDirection=desc&startrow={pager}&_=1686110361253"

    # payload = {}
    # headers = {
    # 'Accept': '*/*',
    # 'Accept-Language': 'en-US,en;q=0.9,bn;q=0.8',
    # 'Connection': 'keep-alive',
    # 'Cookie': 'CookieScriptConsent={"action":"reject","categories":"[]","key":"8f8dfd95-4925-4207-8200-6a3b1993aadf"}; _fbp=fb.1.1685859892012.639773426; _ga=GA1.1.541051862.1685859896; JSESSIONID=w3~923EAB3B12944DDF14C1EE0943765576; _ga_HQJ22DPD35=GS1.1.1686110363.3.0.1686110363.0.0.0',
    # 'Referer': 'https://jobs.issworld.com/search/?utm_source=CorpSite&utm_campaign=All%20Jobs%20Link',
    # 'Sec-Fetch-Dest': 'empty',
    # 'Sec-Fetch-Mode': 'cors',
    # 'Sec-Fetch-Site': 'same-origin',
    # 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    # 'X-CSRF-Token': 'b5ed7ef8-9c47-4eba-b636-6fd58b896f1f',
    # 'X-Requested-With': 'XMLHttpRequest',
    # 'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    # 'sec-ch-ua-mobile': '?0',
    # 'sec-ch-ua-platform': '"macOS"'
    # }

    # response = requests.request("GET", url, headers=headers, data=payload)

    res = session.get(url, headers=header)

    if res.status_code == 200:
        return res.text
    else:
        print(f'Bad Response: {res.status_code}')
        return None

In [ ]:
def getData(job_soup, job_list):
    print('20 Jobs Scraping Started---')
    for job_tag in job_soup:
        job_title = job_tag.select_one('div.tiletitle span.title > a').string.replace('\n', '').strip()
        job_category = job_tag.select_one('div[id$="section-department-value"]').string.replace('\n', '').strip()
        job_loc_1 = job_tag.select_one('div[id$="section-multilocation-value"]').string.replace('\n', '').strip()
        job_loc_2 = job_tag.select_one('div[id$="section-customfield3-value"]').string.replace('\n', '').strip()
        job_location = f'{job_loc_1}, {job_loc_2}'
        job_posted = job_tag.select_one('div[id$="section-date-value"]').string.replace('\n', '').strip()

        base_url = 'https://jobs.issworld.com'
        job_url_tag = job_tag.select_one('div.tiletitle span.title > a')
        job_url_text = job_url_tag['href']
        job_url = parse.urljoin(base_url, job_url_text)

        time_stamp, date_string = getDate(job_posted)

        data_dict = {
            'title': job_title,
            'category': job_category,
            'location': job_location,
            'posted': date_string,
            'job_url': job_url
        }

        print(data_dict)
        job_list.append(data_dict)

    return time_stamp

In [ ]:
job_list = []

for pager in range(0, 650, 20):
    html_res = get_job(pager, header)

    if not html_res:
        print(f'return type -> {html_res}')
    else:
        html = html_res
        soup = BeautifulSoup(html, 'lxml')
        job_tag_list = soup.select('li.job-tile')
        # lowest_stamp = today_stamp() - 259200
        # print(f'lowest_stamp: {lowest_stamp}')
        time_stamp = getData(job_tag_list, job_list)
        print(f'last job time stamp: {time_stamp}')
        



print(len(job_list))

In [ ]:
with open('./data/json/iss_jobs.json', mode='r') as json_file:
    json_data = json_file.read()
    job_list = json.loads(json_data)

In [ ]:
new_job_list = []

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

for job in job_list:
    job_url = job['job_url']

    session = requests.Session()

    res = session.get(job_url, headers=header)
    
    if res.status_code == 200:
        html = res.text

        soup = BeautifulSoup(html, 'lxml')
        job_description = soup.select_one('span.jobdescription').text
        job_description = job_description.replace('\n', '').strip()

        job['description'] = job_description

        new_job_list.append(job)

    else:
        print(f'Bad Response: {res.status_code}')